In [1]:
import pandas as pd
import os
import assets.helper as b3


In [17]:
file_path = 'datasets/nsd_links.zip'

In [18]:
nsd = pd.read_pickle(file_path)  # Try to read the file as a pickle.


In [20]:
nsd.columns

Index(['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor',
       'auditor_rt', 'protocolo', 'envio', 'url', 'nsd'],
      dtype='object')

In [ ]:
b3_companies.fillna('', inplace=True)


In [ ]:
b3_companies.reset_index(drop=True, inplace=True)


In [ ]:
b3_companies.drop_duplicates(inplace=True)


In [ ]:
b3_companies

## PDF Download

In [1]:
import assets.helper as b3
import requests
import json
import base64
from google.cloud import storage
from io import BytesIO

# Set the required properties
codigoInstituicao = 2
numeroProtocolo = 1043145
token = '6LdVyiwaAAAAABobBnLknCD5VGGkmH9snlJBxCyr'
versaoCaptcha = 'V3'


# Send the request
base_url = 'https://www.rad.cvm.gov.br/ENET/'
url = base_url + "frmExibirArquivoIPEExterno.aspx/ExibirPDF"
headers = {"Content-Type": "application/json; charset=utf-8"}

for numeroProtocolo in range (1043146, 1043146+2):
    # # Define the JSON payload
    # data = {
    #     "codigoInstituicao": codigoInstituicao,
    #     "numeroProtocolo": numeroProtocolo,
    #     "token": token,
    #     "versaoCaptcha": versaoCaptcha, 
    # }
    # response = requests.post(url, headers=headers, data=json.dumps(data))
    # # Get the base64-encoded PDF data from the response
    # pdf_data = response.json()['d']

    # # Decode base64-encoded PDF data
    # pdf_bytes = base64.b64decode(pdf_data)

    # # Save PDF data to file
    # with open(f"{numeroProtocolo}.pdf", "wb") as f:
    #     f.write(pdf_bytes)

    
    url = f"https://www.rad.cvm.gov.br/ENET/frmDownloadDocumento.aspx?Tela=ext&numSequencia=567876&numVersao=1&numProtocolo={numeroProtocolo}&descTipo=IPE&CodigoInstituicao=1"
    response = requests.get(url)

    # Save PDF file to Google Cloud Service
    # GCS configuration
    destination_blob_name = f'{numeroProtocolo}.pdf'

    # Initialize GCS client
    client = storage.Client.from_service_account_json(b3.json_key_file)
    bucket = client.get_bucket(b3.bucket_name)

    # Upload the PDF file to GCS
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_string(response.content, content_type='application/pdf')
    # blob.upload_from_string(pdf_bytes, content_type='application/pdf')


    print(numeroProtocolo)



1043146
1043147


## BS4 NSD Links

In [73]:
import requests
from bs4 import BeautifulSoup

empty = 0
nsd_file = 'datasets/nsd_links.zip'
cols_nsd = ['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor', 'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd']
try:
    nsd = pd.read_pickle(nsd_file)
except Exception as e:
    print(e)
    nsd = pd.DataFrame(columns=cols_nsd)

try:
    start = int(max(nsd['nsd'])) + 1
except:
    start = 1

for n in range(start, 10000):
    try:
        nsd_url = f'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento={n}&CodigoTipoInstituicao=1'

        # Getting the HTML content from the URL
        response = requests.get(nsd_url)
        html_content = response.text

        # Parsing the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extracting company
        nomeCompanhia_tag = soup.find('span', {'id': 'lblNomeCompanhia'})
        company = nomeCompanhia_tag.text.strip()

        # Extracting dri and dri2
        nomeDRI_tag = soup.find('span', {'id': 'lblNomeDRI'})
        dri = nomeDRI_tag.text.strip().split(' - ')[0]
        dri2 = nomeDRI_tag.text.strip().split(' - ')[-1].replace('(', '').replace(')', '')

        # Extracting 'FCA', data and versao
        descricaoCategoria_tag = soup.find('span', {'id': 'lblDescricaoCategoria'})
        descricaoCategoria = descricaoCategoria_tag.text.strip()
        versao = descricaoCategoria.split(' - ')[-1]
        data = descricaoCategoria.split(' - ')[1]
        dre = descricaoCategoria.split(' - ')[0]

        # Extracting auditor
        lblAuditor_tag = soup.find('span', {'id': 'lblAuditor'})
        auditor = lblAuditor_tag.text.strip().split(' - ')[0]

        # Extracting auditor_rt
        lblResponsavelTecnico_tag = soup.find('span', {'id': 'lblResponsavelTecnico'})
        auditor_rt = lblResponsavelTecnico_tag.text.strip()

        # Extracting protocolo
        lblProtocolo_tag = soup.find('span', {'id': 'lblProtocolo'})
        protocolo = lblProtocolo_tag.text.strip()

        # Extracting '2010' and envio
        lblDataDocumento_tag = soup.find('span', {'id': 'lblDataDocumento'})
        lblDataDocumento = lblDataDocumento_tag.text.strip()

        lblDataEnvio_tag = soup.find('span', {'id': 'lblDataEnvio'})
        envio = lblDataEnvio_tag.text.strip()

        # cancelamento
        cancelamento_tag = soup.find('span', {'id': 'lblMotivoCancelamentoReapresentacao'})
        cancelamento = cancelamento_tag.text.strip()

        # url
        url = nsd_url

        # company
        company_line = [company, dri, dri2, dre, data, versao, auditor, auditor_rt, cancelamento, protocolo, envio, url, n]
        df = pd.DataFrame([company_line], columns=cols_nsd)
        nsd = pd.concat([nsd, df])
        print(n, company_line)
        empty = 0

    except Exception as e:
        empty += 1
        print(empty, 'empty line', e)

        if empty == 2000:
            print('too much empty lines, aborting...')
            break

    if n % 2 == 0:
        nsd.to_pickle(nsd_file)
        print('partial save')

nsd.to_pickle(nsd_file)
print('final save')


45 ['MARFRIG GLOBAL FOODS S.A.', 'Ricardo Florence dos Santos', 'FCA V2', 'Formulário Cadastral', '2010', 'V1', 'BDO Trevisan Auditores Independentes', 'José Luiz Sanches', '', '020788FCA000020100100000045-80', '17/03/2010 17:03:28', 'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=45&CodigoTipoInstituicao=1', 45]
1 empty line list index out of range
partial save
2 empty line list index out of range
48 ['RENOVA ENERGIA S.A. - EM RECUPERAÇÃO JUDICIAL', 'Vasco de Freitas Barcellos Neto', 'FCA V1', 'Formulário Cadastral', '2010', 'V1', '', '', '', '021636FCA000020100100000048-86', '18/03/2010 13:21:56', 'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=48&CodigoTipoInstituicao=1', 48]
partial save
1 empty line list index out of range
2 empty line list index out of range
partial save
3 empty line list index out of range
4 empty line list index out of range


KeyboardInterrupt: 

In [74]:
nsd

,company,dri,dri2,dre,data,versao,auditor,auditor_rt,cancelamento,protocolo,...,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Formulário Cadastral,2010,V1,,,,021636FCA000020100100000048-86,18/03/2010 13:21:56,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,48
0,MARFRIG GLOBAL FOODS S.A.,Ricardo Florence dos Santos,FCA V2,Formulário Cadastral,2010,V1,BDO Trevisan Auditores Independentes,José Luiz Sanches,,020788FCA000020100100000045-80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,RENOVA ENERGIA S.A. - EM RECUPERAÇÃO JUDICIAL,Vasco de Freitas Barcellos Neto,FCA V1,Formulário Cadastral,2010,V1,,,,021636FCA000020100100000048-86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# nsd = pd.concat([nsd, df])


In [66]:
nsd = pd.DataFrame(columns=cols_nsd)

In [72]:
df

,company,dri,dri2,dre,data,versao,auditor,auditor_rt,cancelamento,protocolo,envio,url,nsd
0,RENOVA ENERGIA S.A. - EM RECUPERAÇÃO JUDICIAL,Vasco de Freitas Barcellos Neto,FCA V1,Formulário Cadastral,2010,V1,,,,021636FCA000020100100000048-86,18/03/2010 13:21:56,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,48


In [68]:
nsd = pd.concat([nsd, df.T])
nsd

,company,dri,dri2,dre,data,versao,auditor,auditor_rt,cancelamento,protocolo,...,3,4,5,6,7,8,9,10,11,12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Formulário Cadastral,2010,V1,,,,021636FCA000020100100000048-86,18/03/2010 13:21:56,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,48


In [13]:
nsd = pd.read_pickle('datasets/nsd_links.zip')


In [15]:
nsd.columns

Index(['company', 'dri', 'dri2', 'dre', 'data', 'versao', 'auditor',
       'auditor_rt', 'cancelamento', 'protocolo', 'envio', 'url', 'nsd'],
      dtype='object')